In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn as sk

class ProcesadorCobros:
    def __init__(self, path_csv, top_n_bancos=4):
        self.path_csv = path_csv
        self.top_n_bancos = top_n_bancos
        self.df = None
        self.Cobradores = None
        self.historial_por_grupo = None
        self.predicciones = None
        self.bancos = None
        self.top_bancos = None
        self.dicccionario = {0: 0, 4: 1, 26: 1}

    def cargar_y_preprocesar(self):
        self.df = pd.read_csv(self.path_csv)
        self.df = self.df.sort_values(by=['idCredito'])
        if 'fechaCobroBanco' in self.df.columns:
            self.df = self.df.drop(columns=['fechaCobroBanco'])
        self.df = self.df.fillna(0)

    def crear_orden(self):
        self.df['orden'] = self.df.groupby('idCredito')['consecutivoCobro'].transform(lambda x: x - x.min())

    def procesar_bancos(self):
        self.top_bancos = self.df['idBanco'].value_counts().nlargest(self.top_n_bancos).index
        # Diccionario de bancos populares
        self.bancos = {b: i for i, b in enumerate(self.top_bancos)}
        # El resto como 4
        self.bancos.update({x: 4 for x in self.df['idBanco'].unique() if x not in self.bancos})
        # Reemplazo de bancos no populares por 1
        self.df['idBanco'] = self.df['idBanco'].apply(lambda x: x if x in self.top_bancos else 1)
        self.df = pd.get_dummies(self.df, columns=['idBanco'], prefix='banco')

    def agrupar_por_credito(self):
        self.Cobradores = self.df.groupby('idCredito')
        self.historial_por_grupo = [grupo.values.tolist() for _, grupo in self.Cobradores]

    def calcular_predicciones(self):
        self.predicciones = self.Cobradores['idRespuestaBanco'].last().reset_index()
        self.predicciones.rename(columns={'idRespuestaBanco': 'valor_prediccion'}, inplace=True)
        self.predicciones['valor_prediccion'] = self.predicciones['valor_prediccion'].replace(self.dicccionario)
        self.predicciones['valor_prediccion'] = np.where(
            self.predicciones['valor_prediccion'].isin([0, 1]),
            self.predicciones['valor_prediccion'],
            0
        )

    def ejecutar_todo(self):
        self.cargar_y_preprocesar()
        self.crear_orden()
        self.procesar_bancos()
        self.agrupar_por_credito()
        self.calcular_predicciones()
        return self
    
    def obtener_X_y_splits(df, predicciones, test_size=0.2, val_size=0.1, random_state=42):

        # Unir el dataframe con las predicciones para asegurar correspondencia
        df_agrupado = df.groupby('idCredito').last().reset_index()
        df_agrupado = df_agrupado.merge(predicciones, on='idCredito')

        # Definir X y y
        X = df_agrupado.drop(['idCredito', 'valor_prediccion', 'idRespuestaBanco'], axis=1, errors='ignore')
        y = df_agrupado['valor_prediccion']

        # Split train+val y test
        X_trainval, X_test, y_trainval, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, stratify=y
        )
        # Split train y val
        val_relative_size = val_size / (1 - test_size)
        X_train, X_val, y_train, y_val = train_test_split(
            X_trainval, y_trainval, test_size=val_relative_size, random_state=random_state, stratify=y_trainval
        )

        return (X_train, y_train), (X_val, y_val), (X_test, y_test)
# Ejemplo de uso:
procesador = ProcesadorCobros('Data/ListaCobroDetalle2025.csv')
procesador.ejecutar_todo()
print(procesador.df.head())
print(procesador.predicciones.head())

        idListaCobro  idCredito  consecutivoCobro  montoExigible  montoCobrar  \
27854         156277       9872          41468078          969.2        969.2   
112144        156458       9872          41558280          969.2        969.2   
223989        156727       9872          41691792          484.6        484.6   
160606        156566       9872          41613223          969.2        969.2   
221225        156711       9872          41688488          484.6        484.6   

        montoCobrado  idRespuestaBanco   orden  banco_1  banco_2  banco_12  \
27854            0.0               4.0       0    False     True     False   
112144           0.0               4.0   90202    False     True     False   
223989           0.0               4.0  223714    False     True     False   
160606           0.0               4.0  145145    False     True     False   
221225           0.0               4.0  220410    False     True     False   

        banco_21  banco_72  
27854      Fals

In [3]:
# Crear un diccionario donde la llave es el idCredito y el valor es el historial (lista de filas, excluyendo idCredito)
columnas_sin_id = [col for col in procesador.df.columns if col != 'idCredito']
historial_por_id = {
    id_credito: grupo[columnas_sin_id].values.tolist()
    for id_credito, grupo in procesador.df.groupby('idCredito')
}
# Ejemplo: mostrar el historial del primer idCredito
primer_id = list(historial_por_id.keys())[0]
print(f"Historial para idCredito {primer_id}:")
for fila in historial_por_id[primer_id]:
    print(fila)
# Si quieres todos los historiales, puedes usar historial_por_id directamente.

Historial para idCredito 9872:
[156277, 41468078, 969.2, 969.2, 0.0, 4.0, 0, False, True, False, False, False]
[156458, 41558280, 969.2, 969.2, 0.0, 4.0, 90202, False, True, False, False, False]
[156727, 41691792, 484.6, 484.6, 0.0, 4.0, 223714, False, True, False, False, False]
[156566, 41613223, 969.2, 969.2, 0.0, 4.0, 145145, False, True, False, False, False]
[156711, 41688488, 484.6, 484.6, 0.0, 4.0, 220410, False, True, False, False, False]
[156625, 41656948, 484.6, 484.6, 0.0, 4.0, 188870, False, True, False, False, False]
[156525, 41593527, 969.2, 969.2, 0.0, 4.0, 125449, False, True, False, False, False]
[156692, 41684150, 484.6, 484.6, 0.0, 4.0, 216072, False, True, False, False, False]
[156602, 41641011, 969.2, 969.2, 0.0, 4.0, 172933, False, True, False, False, False]
[156378, 41524210, 969.2, 969.2, 0.0, 4.0, 56132, False, True, False, False, False]
[156311, 41483305, 969.2, 969.2, 0.0, 4.0, 15227, False, True, False, False, False]
[156749, 41702921, 484.6, 484.6, 0.0, 4.0,